In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

# Basics of combinatorial circuits using Chisel3 

In this notebook you will get introduced to the basics of Chisel. This will include 
- Modules
 - IOs  
- Chisel types/literals and operators
 - UInt, SInt, Bool.
 - +, -, === etc.
- Wires
- Registers
- Multiplexers



## Module and IO
The first Chisel keyword to be introduced is *Module* and *IO*.  
When declaring a class extending Module, it means that this code will be mapped to a Verilog module. Verilog modules are equivalent to components in VHDL. You start a Module description by declaring the inputs and outputs of the module. This is done using the special keyword *IO*. Declaring IO is equivalent to declaring the entity of a component in VHDL. All signals declared in IO must be specified as either an input or an output, this is done by wrapping the signals in the Input() and Output() apply methods (see code below). The IO declared in the empty module *HardwareModule* has signals from the three basic chisel data types; UInt - unsigned integer, SInt - signed integer and Bool - boolean. You specify the with of the signal bus explicitly using the X.W with x being an integer larger than 0. You cannot set the with of a Bool as it represents a single wire. If you need a bus and don't interpret it as an integer use UInt anyway. Inspect the code below. 

In [ ]:
class HardwareModule extends Module {   // Declare a class extending Module to declare the equivilent of a VHDL component
    val io = IO(new Bundle {            // Declare in and outputs of the module. Equivilent to an entity declaration in VHDL - by convention named "io"
        val input1 = Input(UInt(8.W))   // Declaration of input as a 8-bit bus that can be interpredted as a unsigned integer.
        val input2 = Input(SInt(16.W))  // Declaration of input as a 16-bit bus that can be interpredted as a signed integer.
        
        val output1 = Output(UInt(8.W)) // Declaration of output as a 8-bit bus that can be interpredted as a unsigned integer.
        val output2 = Output(Bool())    // Declaration of output signal. Bool is a signle wire that can either be 0 or 1.
    })
}

### Body of modules
In the body of the module, we describe the circuit of the module. This is equivalent to the architecture declaration in VHDL. We introduce this by making a module that takes three inputs a, b and c and produces the output z = (a + b) * c. This will be described in two steps, to introduce declaration of wires internal to a module, and signal value assignment. wires are declared using Wire(). See code below and read the comments. 

In [ ]:
class AddMult extends Module {
    val io = IO(new Bundle {
        val a = Input(SInt(32.W))
        val b = Input(SInt(32.W))
        val c = Input(SInt(32.W))
        
        val z = Output(SInt(32.W))
    })
    
    val sum = Wire(SInt(32.W))     
    // sum is declared as an internal wire of the SInt type, is a bus that is 32-bits wide.
    val product = Wire(SInt(32.W)) 
    
    sum := io.a + io.b 
    // The line above is an assignment "sum" with the addition of inputs a and b. Assignments are 
    // written with ":=" notice that the inputs are referred to by the prefix "io." which correspond 
    // to the name of the IO of this module. Remember that we are describing hardware, which means 
    // every assignment is evaluated concurrently. This means that you can only assign one value to a
    // wire, otherwise you are describing a short circuit.
    product := sum * io.c 
    io.z := product
    
}
// the following line prints the verilog code that this module generates. This will might give the reader a better
// understanding of whats going on.
println(getVerilog(new AddMult))

## Operaters
Operators such as + and * looks identical for scala and chisel. They are interpreded as hardware if the operand are Chisel types, which is the case for the operators used in the code above. All operands is an assignment must be Chisel types. If you need a satatic value as operand for an operator a postfix is used:

```scala
42.U    // .U specifies the Chisel type UInt
42.S    // .S specifies the Chisel type SInt
true.B  // .B specifies the Chisel type Bool

val a = 42 + 42     // OK - Scala Addition
val a = 42.U + 42.U // OK - Chisel adder unit of adding two static signals.
val a = 42 + 42.U   // Not Cool! - You can't mix scala and chisel types in arithmetic expressions
```

For a list of all Chisel operators please see the official [Chisel3 Cheatsheet](https://www.chisel-lang.org/doc/chisel-cheatsheet3.pdf/)
